In [1]:
import os
from time import time

import numpy as np

from sklearn.linear_model import RidgeClassifier

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader

from model_loading_utils import pick_model
from features import fast_conv_features, get_random_features, decoding, dummy_predict_GPU
from dataset import CatsAndDogs

In [2]:
dataset_path = "/home/luca/datasets/cats-and-dogs-breeds-classification-oxford-dataset"

images_path = os.path.join(dataset_path, "images", "images")
annotation_path = os.path.join(dataset_path, "annotations", "annotations")

In [3]:
batch_size = 32 
num_workers = 12

train_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

train_dataset = CatsAndDogs(dataset_path, mode="trainval", transform=train_transform)
test_dataset = CatsAndDogs(dataset_path, mode="test", transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [4]:
model_name, model_options = "resnet50", "noavgpool"
device = "cuda:0"
encode_type = "positive"
model_dtype = "float16"

model, output_size = pick_model(model_name=model_name, model_options=model_options, device=device, dtype=model_dtype)

resnet50 model loaded successfully.


In [5]:
enc_train_features, train_labels, train_conv_time, train_encode_time = fast_conv_features(train_loader, model,
                                                                                          output_size,
                                                                                          device=device,
                                                                                          encode_type=encode_type,
                                                                                          dtype=model_dtype)

enc_test_features, test_labels, test_conv_time, test_encode_time = fast_conv_features(test_loader, model,
                                                                                      output_size, device=device,
                                                                                      encode_type=encode_type,
                                                                                      dtype=model_dtype)

In [6]:
n_components = 0
train_proj_time, train_random_features = get_random_features(enc_train_features, n_components)
test_proj_time, test_random_features = get_random_features(enc_test_features, n_components)

In [7]:
train_decode_time, dec_train_random_features = decoding(train_random_features, decode_type=None)
test_decode_time, dec_test_random_features = decoding(test_random_features, decode_type=None)

In [ ]:
alpha_exp_min, alpha_exp_max, alpha_space = 2, 5, 3

alpha_mant = np.linspace(1, 9, alpha_space)
alphas = np.concatenate([alpha_mant * 10 ** i for i in range(alpha_exp_min, alpha_exp_max + 1)])

for alpha in alphas:
    clf = RidgeClassifier(alpha=alpha)
    since = time()
    clf.fit(dec_train_random_features, train_labels)
    fit_time = time() - since
    
    
    train_accuracy = clf.score(dec_train_random_features, train_labels) * 100
    test_accuracy = clf.score(dec_test_random_features, test_labels) * 100
    
    print("alpha = {0:.2e}\tTrain acc = {1:2.2f}\tTest acc = {2:2.2f}".format(alpha, train_accuracy, test_accuracy))

alpha = 1.00e+02	Train acc = 100.00	Test acc = 85.06
alpha = 5.00e+02	Train acc = 100.00	Test acc = 85.01
alpha = 9.00e+02	Train acc = 100.00	Test acc = 85.04
alpha = 1.00e+03	Train acc = 100.00	Test acc = 85.04
alpha = 5.00e+03	Train acc = 100.00	Test acc = 85.31
alpha = 9.00e+03	Train acc = 100.00	Test acc = 85.20
alpha = 1.00e+04	Train acc = 100.00	Test acc = 85.23
alpha = 5.00e+04	Train acc = 99.89	Test acc = 84.57
alpha = 9.00e+04	Train acc = 99.54	Test acc = 83.54
alpha = 1.00e+05	Train acc = 99.48	Test acc = 83.46
